# NewCo Data Explore
**Goal:** Take a look at some of the data that has come in from Kevin Gurney's team for onroad surface emissions.

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import statsmodels
import numpy as np
import geopandas as gpd
from keplergl import KeplerGl
import geopandas as gpd
import os

pd.options.mode.chained_assignment = None  # default='warn'

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pio.templates.default = "none"
%config InlineBackend.figure_format ='retina'

## Data Read-in and Cleaning
Data exists in this google drive link: https://drive.google.com/drive/u/0/folders/15NItZvPFgNxUR-N4AC3hWZN2JqDfjm_L

Given file size restraints, this folder is downloaded locally but has been excluded from the github branch. Update path below to whichever local path is used

In [2]:
local_path = './raw_data_not_on_github/'
block_df = pd.read_csv(local_path + 'nonpoint_block_group.csv',
                       dtype = {'FIPS': str, 'STCOFIPS': str, 'STATE_FIPS': str}) # Read FIPs as strings to handle leading zeros
block_df.head()
block_df.dtypes
#Note: reading in the geodataset takes a while
# geo_df = gpd.read_file(local_path + 'Hestia_LA_onroad_annual.GeoJSON')
# geo_df.head()

,STATE_FIPS,STCOFIPS,FIPS,Vulcan.sector.description,Fuel.type,BG.CO2.tC_2010,BG.CO2.tC_2011,BG.CO2.tC_2012,BG.CO2.tC_2013,BG.CO2.tC_2014,BG.CO2.tC_2015
0,1,1001,10010201001,Commercial/Institutional,Coal,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,1001,10010201001,Commercial/Institutional,Electricity,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1,1001,10010201001,Commercial/Institutional,Natural Gas,29.623095,27.535945,23.593311,27.728820,30.304086,27.585511
3,1,1001,10010201001,Commercial/Institutional,Petroleum,7.349044,7.749572,6.929595,5.325909,4.886748,8.776120
4,1,1001,10010201001,Industrial,Coal,0.916789,0.865634,0.971662,1.018128,1.162522,0.925555


STATE_FIPS                    object
STCOFIPS                      object
FIPS                          object
Vulcan.sector.description     object
Fuel.type                     object
BG.CO2.tC_2010               float64
BG.CO2.tC_2011               float64
BG.CO2.tC_2012               float64
BG.CO2.tC_2013               float64
BG.CO2.tC_2014               float64
BG.CO2.tC_2015               float64
dtype: object

# Census Block Explore
Let's take a look at this census block dataset first. We can look at all the emissions, for five years, by sector by census block. Let's take a look at aggregate trends before reading in a polygon dataset to put this on a map.

In [8]:
#Let's look at data for a single state just to see what's in here
state_select = '26' # Michigan
state_df = block_df[block_df.STATE_FIPS == state_select]


# Let's look at some aggregations
# Total CO2 by year
t_df = state_df.groupby('STATE_FIPS').sum().reset_index().melt(id_vars = 'STATE_FIPS')
px.bar(t_df,x='variable',y='value',title = 'Tons of CO2 Emissions per Year')

# Total CO2 by fuel type
t_df = state_df.groupby('Fuel.type').sum().reset_index().melt(id_vars = ['Fuel.type'])
px.area(t_df,x='variable',y='value',color = 'Fuel.type',title = 'Emissions by Fuel Type')
printmd('**--> It seems strange that electricity has a zero emissions value (Michigan)**')
# Total CO2 by commercial sector
t_df = state_df.groupby('Vulcan.sector.description').sum().reset_index().melt(id_vars = ['Vulcan.sector.description'])
px.area(t_df,x='variable',y='value',color = 'Vulcan.sector.description',title = 'Emissions by Sector')

# Total CO2 by both sector and fuel type
t_df = state_df.groupby(['Vulcan.sector.description','Fuel.type']).sum().reset_index()
px.bar(t_df,x='BG.CO2.tC_2015',y='Vulcan.sector.description',color='Fuel.type',barmode='group',title='Emissions by Sector and Fuel Type (2015)')


**--> It seems strange that electricity has a zero emissions value (Michigan)**

In [9]:
# Let's also just look at total country emissions to see how it stacks up against other reported datasets
year_list = ['BG.CO2.tC_2010', 'BG.CO2.tC_2011', 'BG.CO2.tC_2012',
       'BG.CO2.tC_2013', 'BG.CO2.tC_2014', 'BG.CO2.tC_2015']
px.bar(block_df[year_list].sum(),title='Annual US Emissions (Tons CO2)')

px.area(block_df.groupby('Fuel.type').sum().reset_index().melt(id_vars = ['Fuel.type']),
        x='variable',y='value',color = 'Fuel.type',title = 'Emissions by Fuel Type')

px.area(block_df.groupby('Vulcan.sector.description').sum().reset_index().melt(id_vars = ['Vulcan.sector.description']),
        x='variable',y='value',color = 'Vulcan.sector.description',title = 'Emissions by Sector')

px.area(block_df.groupby('Vulcan.sector.description').sum().reset_index().melt(id_vars = ['Vulcan.sector.description']),
        x='variable',y='value',color = 'Vulcan.sector.description',title = 'Emissions by Sector (% of Total)',groupnorm='percent')

In [10]:
#Next let's look at some cross-state comparisons for fuel
t_df = block_df.groupby(['STATE_FIPS','Fuel.type']).sum().reset_index().melt(id_vars=['STATE_FIPS','Fuel.type'])
px.bar(t_df,x='value',y='STATE_FIPS',color='Fuel.type',facet_col = 'variable',
       title = 'Emissions by Fuel Type by State by Year')

px.bar(t_df[t_df.variable == 'BG.CO2.tC_2015'],x='value',y='STATE_FIPS',color='Fuel.type',
       title = 'Emissions by Fuel Type by State (2015 Only)',height = 600)

printmd('**--> Looks like there are no electricity emissions in any state**')

#Let's do the same for sector
t_df = block_df.groupby(['STATE_FIPS','Vulcan.sector.description']).sum().reset_index().melt(id_vars=['STATE_FIPS','Vulcan.sector.description'])
px.bar(t_df,x='value',y='STATE_FIPS',color='Vulcan.sector.description',facet_col = 'variable',
       title = 'Emissions by Sector by State by Year')

px.bar(t_df[t_df.variable == 'BG.CO2.tC_2015'],x='value',y='STATE_FIPS',color='Vulcan.sector.description',
       title = 'Emissions by Sector by State (2015 Only)',height = 600)

**--> Looks like there are no electricity emissions in any state**

## Census Block Visualization
Let's try to take a look at how emissions look for the state of California.

In [14]:
#Read in block_df again, this time without dtype definitions
block_df = pd.read_csv(local_path + 'nonpoint_block_group.csv')
block_df.head()
geo_census = gpd.read_file(local_path + 'cb_2020_06_bg_500k.json')
geo_census[geo_census.STATEFP == '06'].head()

,STATE_FIPS,STCOFIPS,FIPS,Vulcan.sector.description,Fuel.type,BG.CO2.tC_2010,BG.CO2.tC_2011,BG.CO2.tC_2012,BG.CO2.tC_2013,BG.CO2.tC_2014,BG.CO2.tC_2015
0,1,1001,1.001020e+10,Commercial/Institutional,Coal,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,1001,1.001020e+10,Commercial/Institutional,Electricity,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1,1001,1.001020e+10,Commercial/Institutional,Natural Gas,29.623095,27.535945,23.593311,27.728820,30.304086,27.585511
3,1,1001,1.001020e+10,Commercial/Institutional,Petroleum,7.349044,7.749572,6.929595,5.325909,4.886748,8.776120
4,1,1001,1.001020e+10,Industrial,Coal,0.916789,0.865634,0.971662,1.018128,1.162522,0.925555


,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,AFFGEOID,GEOID,NAME,NAMELSAD,LSAD,ALAND,AWATER,geometry
0,06,075,980401,1,1500000US060759804011,060759804011,1,Block Group 1,BG,419323,247501289,"POLYGON ((-123.01392 37.70036, -123.01278 37.6..."
1,06,059,062605,3,1500000US060590626053,060590626053,3,Block Group 3,BG,265453,845383,"POLYGON ((-117.78250 33.53954, -117.77929 33.5..."
2,06,001,428700,2,1500000US060014287002,060014287002,2,Block Group 2,BG,5877911,4050342,"POLYGON ((-122.33672 37.80034, -122.33567 37.7..."
3,06,059,042106,2,1500000US060590421062,060590421062,2,Block Group 2,BG,1953370,766370,"POLYGON ((-117.64572 33.44083, -117.64559 33.4..."
4,06,037,575103,2,1500000US060375751032,060375751032,2,Block Group 2,BG,263278,0,"POLYGON ((-118.15675 33.78623, -118.15675 33.7..."


In [15]:
geo_census['GEOID_int'] = geo_census['GEOID'].astype(int)
geo_census.head()

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,AFFGEOID,GEOID,NAME,NAMELSAD,LSAD,ALAND,AWATER,geometry,GEOID_int
0,06,075,980401,1,1500000US060759804011,060759804011,1,Block Group 1,BG,419323,247501289,"POLYGON ((-123.01392 37.70036, -123.01278 37.6...",60759804011
1,06,059,062605,3,1500000US060590626053,060590626053,3,Block Group 3,BG,265453,845383,"POLYGON ((-117.78250 33.53954, -117.77929 33.5...",60590626053
2,06,001,428700,2,1500000US060014287002,060014287002,2,Block Group 2,BG,5877911,4050342,"POLYGON ((-122.33672 37.80034, -122.33567 37.7...",60014287002
3,06,059,042106,2,1500000US060590421062,060590421062,2,Block Group 2,BG,1953370,766370,"POLYGON ((-117.64572 33.44083, -117.64559 33.4...",60590421062
4,06,037,575103,2,1500000US060375751032,060375751032,2,Block Group 2,BG,263278,0,"POLYGON ((-118.15675 33.78623, -118.15675 33.7...",60375751032


In [16]:
geo_census['GEOID_int'] = geo_census['GEOID'].astype(int)
cal_df = block_df[block_df.STATE_FIPS == 6].groupby('FIPS').sum().reset_index().merge(
                geo_census[geo_census.STATEFP == '06'][['STATEFP','GEOID_int','geometry']],
                how = 'left',
                left_on = 'FIPS',
                right_on = 'GEOID_int')
cal_df.head()

,FIPS,STATE_FIPS,STCOFIPS,BG.CO2.tC_2010,BG.CO2.tC_2011,BG.CO2.tC_2012,BG.CO2.tC_2013,BG.CO2.tC_2014,BG.CO2.tC_2015,STATEFP,GEOID_int,geometry
0,6.001400e+10,72,72012,1296.173278,1334.488979,1249.782993,1267.799972,1057.398231,1075.518062,06,6.001400e+10,"POLYGON ((-122.24691 37.88535, -122.24683 37.8..."
1,6.001400e+10,72,72012,567.358945,579.418184,554.866379,567.640494,495.779089,501.255750,06,6.001400e+10,"POLYGON ((-122.25250 37.85083, -122.25235 37.8..."
2,6.001400e+10,72,72012,378.041980,387.959318,365.656293,370.788950,311.949517,316.932351,06,6.001400e+10,"POLYGON ((-122.25620 37.84469, -122.25742 37.8..."
3,6.001400e+10,72,72012,486.799790,500.799721,469.648210,476.435270,398.332344,405.059436,06,6.001400e+10,"POLYGON ((-122.25186 37.84475, -122.25141 37.8..."
4,6.001400e+10,72,72012,563.445531,577.903856,545.083289,552.724401,465.622111,473.010510,06,6.001400e+10,"POLYGON ((-122.25807 37.83829, -122.26049 37.8..."


In [17]:
cal_df[['FIPS','BG.CO2.tC_2015','geometry']].to_csv('cal_map.csv')

In [19]:
#Let's look at distribution of 2015 emissions for California
px.histogram(cal_df,x='BG.CO2.tC_2015',title = 'Histogram of Census Block Emissions (California)')

px.box(cal_df,x='BG.CO2.tC_2015', title = 'Box Plot of Emissions by Census Block')

In [25]:
cal_df.sort_values(by='BG.CO2.tC_2015',ascending=False).head(20)

,FIPS,STATE_FIPS,STCOFIPS,BG.CO2.tC_2010,BG.CO2.tC_2011,BG.CO2.tC_2012,BG.CO2.tC_2013,BG.CO2.tC_2014,BG.CO2.tC_2015,STATEFP,GEOID_int,geometry
10203,6.053001e+10,72,72636,874258.667318,968414.814708,916342.041884,972629.956582,939824.278551,941844.621402,NaN,NaN,None
10219,6.053010e+10,72,72636,271488.721455,299836.413935,284316.769472,301714.919700,291724.757412,292302.911311,06,6.053010e+10,"POLYGON ((-121.77227 36.88385, -121.77000 36.8..."
10281,6.053011e+10,72,72636,219225.457478,242858.114103,229687.528944,243753.064493,235350.133891,235881.962200,NaN,NaN,None
10361,6.053013e+10,72,72636,215807.977410,238963.420432,226126.016741,239971.635894,231791.911466,232299.695882,NaN,NaN,None
10356,6.053013e+10,72,72636,202943.222325,224908.228743,212680.974466,225711.171914,217943.368018,218434.954330,06,6.053013e+10,"POLYGON ((-121.88046 36.59159, -121.87967 36.5..."
10217,6.053002e+10,72,72636,144122.551047,159301.329802,150837.709706,160014.309410,154452.635193,154799.550693,NaN,NaN,None
10376,6.053014e+10,72,72636,98733.785865,109397.147333,103484.319010,109828.266366,106084.087542,106317.680484,NaN,NaN,None
10201,6.053001e+10,72,72636,93523.619472,103575.568725,98022.913674,104045.332799,100547.617039,100761.727237,NaN,NaN,None
10237,6.053010e+10,72,72636,77604.230250,85991.866625,81319.096327,86301.643530,83330.722762,83519.051926,06,6.053010e+10,"POLYGON ((-121.75594 36.76951, -121.75975 36.7..."
10291,6.053012e+10,72,72636,60716.889896,67003.221147,63450.061473,67226.821066,64725.168007,64887.179664,NaN,NaN,None


In [ ]:
cal_df.sort_values(by='BG.CO2.tC_2015',ascending=False).head(5)

In [26]:
block_df[block_df.FIPS == 60530132002]

,STATE_FIPS,STCOFIPS,FIPS,Vulcan.sector.description,Fuel.type,BG.CO2.tC_2010,BG.CO2.tC_2011,BG.CO2.tC_2012,BG.CO2.tC_2013,BG.CO2.tC_2014,BG.CO2.tC_2015
2292012,6,6053,6.053013e+10,Commercial/Institutional,Coal,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2292013,6,6053,6.053013e+10,Commercial/Institutional,Electricity,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2292014,6,6053,6.053013e+10,Commercial/Institutional,Natural Gas,634.696519,628.528165,647.093356,655.173348,612.353149,612.485936
2292015,6,6053,6.053013e+10,Commercial/Institutional,Petroleum,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2292016,6,6053,6.053013e+10,Industrial,Coal,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2292017,6,6053,6.053013e+10,Industrial,Electricity,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2292018,6,6053,6.053013e+10,Industrial,Natural Gas,3420.346178,3333.596285,3498.886302,3693.496924,3726.136186,3700.229803
2292019,6,6053,6.053013e+10,Industrial,Petroleum,198314.599651,220358.876119,207994.520940,220814.636522,213153.667321,213660.579285
2292020,6,6053,6.053013e+10,Residential,Coal,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2292021,6,6053,6.053013e+10,Residential,Electricity,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [24]:
block_df[block_df.STATE_FIPS == 6].sort_values(by='BG.CO2.tC_2015',ascending=False).head(30)

,STATE_FIPS,STCOFIPS,FIPS,Vulcan.sector.description,Fuel.type,BG.CO2.tC_2010,BG.CO2.tC_2011,BG.CO2.tC_2012,BG.CO2.tC_2013,BG.CO2.tC_2014,BG.CO2.tC_2015
2290183,6,6053,6.053001e+10,Industrial,Petroleum,851931.795282,946630.925149,893515.383866,948588.808125,915678.355461,917855.978388
2290375,6,6053,6.053010e+10,Industrial,Petroleum,257763.380719,286415.871448,270345.052674,287008.255178,277050.756718,277709.625715
2291119,6,6053,6.053011e+10,Industrial,Petroleum,213423.247119,237146.972314,223840.635637,237637.454937,229392.832879,229938.364057
2292079,6,6053,6.053013e+10,Industrial,Petroleum,209458.693510,232741.726411,219682.568455,233223.097821,225131.627992,225667.025375
2292019,6,6053,6.053013e+10,Industrial,Petroleum,198314.599651,220358.876119,207994.520940,220814.636522,213153.667321,213660.579285
2290351,6,6053,6.053002e+10,Industrial,Petroleum,137354.402516,152622.458567,144058.799507,152938.122154,147632.068797,147983.160396
2292259,6,6053,6.053014e+10,Industrial,Petroleum,96380.027288,107093.449150,101084.426659,107314.946711,103591.749216,103838.106211
2290159,6,6053,6.053001e+10,Industrial,Petroleum,90994.165178,101108.904777,95435.675578,101318.024718,97802.885147,98035.475340
2290591,6,6053,6.053010e+10,Industrial,Petroleum,75740.705152,84159.898934,79437.679886,84333.963851,81408.071302,81601.672125
2291239,6,6053,6.053012e+10,Industrial,Petroleum,56683.717052,62984.572017,59450.502354,63114.840761,60925.126987,61070.016240


## GeoJSON Explore
Before we try to map out any of the census data, let's take a look real quick at what's in the GeoJSON dataset they sent over.

In [ ]:
geo_df = gpd.read_file(local_path + 'Hestia_LA_onroad_annual.GeoJSON')
geo_df.head()

In [ ]:
#Let's do some explores of 

In [ ]:
map1 = KeplerGl(data={'data_sample':geo_df[['ca10','geometry']].head(10)})
map1

In [ ]:
map2 = KeplerGl()
map2.add_data()
map2

In [ ]:
with open(local_path + 'Hestia_LA_onroad_annual.GeoJSON', 'r') as f:
    la_geojson = f.read()
    
la_geojson